# Практическое задание №3 по теме "Embedding word2vec fasttext".

### Загружаем библиотеки:

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import time
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec, FastText

### Используем датасет lenta-ru-news.csv:

In [2]:
df = pd.read_csv('/content/drive/MyDrive/lenta-ru-news.csv')

### Удаляем ненужные признаки:

In [3]:
df = df.drop(columns=['url','topic','tags'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739351 entries, 0 to 739350
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   739351 non-null  object
 1   text    739346 non-null  object
dtypes: object(2)
memory usage: 11.3+ MB


In [5]:
df.head()

,title,text
0,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...
1,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...
2,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...
3,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...
4,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...


### Функция предобработки и токенизации :

In [6]:
def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [7]:
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
stopwrd = set(stopwords.words("russian"))
df["tokens"] = df["title"].map(lambda x: clean_text(x, word_tokenize, stopwrd))
df.head()

,title,text,tokens
0,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,"[названы, регионы, россии, самой, высокой, сме..."
1,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,"[австрия, представила, доказательств, вины, ро..."
2,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,"[обнаружено, самое, счастливое, место, планете]"
3,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,"[сша, раскрыли, сумму, расходов, расследование..."
4,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,"[хакеры, рассказали, планах, великобритании, з..."


### Применение Word2Vec:

In [9]:
model = Word2Vec(sentences=df["tokens"].values, vector_size=100, workers=1)

In [10]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(df["tokens"].values, model=model)
len(vectorized_docs), len(vectorized_docs[0])

(739351, 100)

In [11]:
from sklearn.neighbors import KDTree
X = np.vstack(vectorized_docs)
tree = KDTree(X)

In [12]:
test_text = 'островные коты иммигрировали в норвегию'
test_tokenized = clean_text(test_text, word_tokenize, stopwrd)
test_vec = vectorize([test_tokenized], model=model)

_, ind = tree.query(test_vec, k=5)
[df.text.iloc[i] for i in ind[0]]

['Агенты знаменитого британского актера Шона Коннери  (Sean Connery) ведут переговоры с чешским автозаводом Skoda о съемках сэра Шона в рекламе для испанского рынка автомобиля Octavia. Ролик предполагается снимать в Шотландии, и актер предстанет в нем в традиционной шотландской юбке-килте. Возможно, именно это и заставило знаменитого актера благожелательно отнестись к идее участия в такой рекламе: сэр Шон не скрывает своих симпатий к шотландским националистам, и именно это долгое время служило ему препятствием для получения британского рыцарского звания. ВВС напоминает, что в качестве Джеймса Бонда Коннери стал самым знаменитым промоутером британских автомобилей Aston Martin - он разъезжал на них в шести фильмах про агента 007. То, что в последних трех фильмах Бонд (уже не Коннери) ездит на BMW,  воспринималось в Великобритании чуть ли не как национальная катастрофа - и очень повышало продажи немецкого автогиганта.',
 'Метание топора и аркана (для ловли оленей), бег с палкой и гонки на

### Применение FastText: 

In [13]:
modelFT = FastText(sentences=df["tokens"], vector_size=100, min_count=1, window=5, workers=1)

In [14]:
vectorized_docs = vectorize(df["tokens"], model=modelFT)
X = np.vstack(vectorized_docs)
tree = KDTree(X)

test_text = 'островные коты иммигрировали в норвегию'
test_tokenized = clean_text(test_text, word_tokenize, stopwrd)
test_vec = vectorize([test_tokenized], model=modelFT)

_, ind = tree.query(test_vec, k=5)
[df.text.iloc[i] for i in ind[0]]

['Байкеры из клуба «Ночные волки», совершающие мотопробег по Европе, беспрепятственно пересекли границу с Польшей. Об этом сообщила представитель пограничной службы страны Мирослава Александрович, передает сайт «Русский Диалог». «Польские пограничники не проверяли наличие в колонне мотоциклетных клубов, а проверили мотоциклистов по отдельности», — сказала она. По словам Александрович, пограничники узнали у россиян, на какие мероприятия они едут. «Каждый из них заявил, что примет участие в церемонии в честь советских воинов на кладбище в Бранево», — сообщила она. От границы колонна из 200 российских байкеров сопровождалась польскими полицейскими. По пути не было никаких инцидентов. В мотопробеге участвует губернатор Калининградской области Николай Цуканов. «Ночные волки» побывали на месте захоронения советских солдат на кладбище в польском городе Бранево. Всего в мероприятии приняли участие более 500 россиян. Москву представляли посол в Польше Сергей Андреев и генеральный консул в Гдань

**Вывод:** Оценить результат довольно сложно, но обе модели предсказали тексты новостей довольно далекие от тестовой фразы.

---